In [12]:
import spacy
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tanmayvakare/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [27]:
np = spacy.load('en_core_web_sm')

In [40]:
parsed = np("He eats orange and apple.")
len(parsed)

6

In [41]:
spacy.displacy.render(parsed, jupyter=True)

In [50]:
for each in parsed:
    print(f"{each.head}------{each.dep_}------>{each.text}({each.pos_})")
pairs = [(token.head,token.dep_,token.text) for token in parsed]
for pair in pairs:
    print(type(pair[0]))

eats------nsubj------>He(PRON)
eats------ROOT------>eats(VERB)
eats------dobj------>orange(NOUN)
orange------cc------>and(CCONJ)
orange------conj------>apple(NOUN)
eats------punct------>.(PUNCT)
<class 'spacy.tokens.token.Token'>
<class 'spacy.tokens.token.Token'>
<class 'spacy.tokens.token.Token'>
<class 'spacy.tokens.token.Token'>
<class 'spacy.tokens.token.Token'>
<class 'spacy.tokens.token.Token'>


In [62]:
sparta = wn.synsets("sparta")[0]
be = wn.synsets("is")[0]
print(sparta.path_similarity(be))

None


In [64]:
def similarity_word(pair_A, pair_B):
    
    #getting head and dependent texts 
    head_a, head_b = pair_A[0].text, pair_B[0].text
    dep_a, dep_b = pair_A[2].text, pair_B[2].text
    
    if head_a == head_b:
        head = 1
    else:
        try:
            #WordNet synsets for heads
            head_a, head_b = wn.synsets(head_a)[0], wn.synsets(head_b)[0]

            #path based similarity (Li et. al) for head
            head = head_a.path_similarity(head_b)
            
            head = 0 if head is None else head  
        
        except Exception:
            head = 0
    
    if dep_a == dep_b:
        dep = 1
    else:
        try:
            #WordNet synsets for dependent
            dep_a, dep_b = wn.synsets(dep_a)[0], wn.synsets(dep_b)[0]

            #path based similarity (Li et. al) for dependent
            dep = dep_a.path_similarity(dep_b)
            
            dep = 0 if dep is None else dep

        except Exception:
            dep = 0
    
#     print(f"{dep_a}:{dep_b}--{head} {dep}")        
    
    return head + dep
    
similarity_tag = lambda a, b : 1 if a == b else 0

def semantic_similarity(document_1, document_2):
    
    #parsing documets using spaCy English language parser
    tokens_1,tokens_2 = np(document_1), np(document_2)
    
    #seperating dependency pairs and tags from tokens
    pairs_1 = [(token.head,token.dep_,token) for token in tokens_1]
    pairs_2 = [(token.head,token.dep_,token) for token in tokens_2]
    
    score = 0
    
    #calculating score 
    for pair_A in pairs_1:
        
        for pair_B in pairs_2:
            
            score += similarity_word(pair_A, pair_B) * similarity_tag(pair_A[1], pair_B[1])
    
    #averaging score 
    score = score / (len(tokens_1) + len(tokens_2))
    
    return score 

In [70]:
semantic_similarity("I haven't watched television for ages.","It's been a long time since I watched television.")


0.31578947368421045

In [71]:
spacy.displacy.render(np("It's been a long time since I watched television."),jupyter=True)

In [72]:
spacy.displacy.render(np("I haven't watched television for ages."),jupyter=True)